In [ ]:
import os, ast

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import StratifiedKFold

In [ ]:
train_df = pd.read_csv(os.path.join('/kaggle/input/global-wheat-detection/', 'train.csv'))
# Let's expand the bounding box coordinates and calculate the area of all the bboxes
train_df[['x_min','y_min', 'width', 'height']] = pd.DataFrame([ast.literal_eval(x) for x in train_df.bbox.tolist()], index= train_df.index)
train_df = train_df[['image_id', 'bbox', 'source', 'x_min', 'y_min', 'width', 'height']]
train_df['area'] = (train_df['width'] * train_df['height']).astype(np.int64)
train_df['x_max'] = (train_df['x_min'] + train_df['width']).astype(np.int64)
train_df['y_max'] = (train_df['y_min'] + train_df['height']).astype(np.int64)
train_df = train_df.drop(['bbox'], axis=1)
train_df[['x_min', 'y_min', 'width', 'height']] = train_df[['x_min', 'y_min', 'width', 'height']].astype(np.int64)
train_df = train_df[['image_id', 'x_min', 'y_min', 'x_max', 'y_max', 'width', 'height', 'area', 'source']]

# There are some buggy annonations in training images having huge bounding boxes. Let's remove those bboxes
train_df = train_df[train_df['area'] < 100000]

train_df.head()

In [ ]:
# Add a new column to store kfold indices
train_df.loc[:, 'kfold'] = -1
train_df.shape

In [ ]:
image_source = train_df[['image_id', 'source']].drop_duplicates()

# get lists for image_ids and sources
image_ids = image_source['image_id'].to_numpy()
sources = image_source['source'].to_numpy()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1996)
split = skf.split(image_ids, sources) # second arguement is what we are stratifying by

for fold, (train_idx, val_idx) in enumerate(split):
    translated_val_idx = train_df[train_df['image_id'].isin(image_ids[val_idx])].index.values
    print(len(translated_val_idx))
    train_df.loc[translated_val_idx, 'kfold'] = fold

In [ ]:
train_df

In [ ]:
train_df.to_csv('train.csv', index=False)